In [1]:
import sys
from pathlib import Path

In [5]:
src_path = Path("../src").resolve()
sys.path.append(str(src_path))

In [9]:
from sqlmodel import Session, select
from api.db.session import engine
from api.events.models import EventModel


In [12]:
with Session(engine) as session:
    query = select(EventModel).order_by(EventModel.updated_at.asc()).limit(10)
    compiled_query = query.compile(compile_kwargs={"literal_binds": True})
    print(compiled_query)
    print("")
    print(str(query))

SELECT eventmodel.id, eventmodel.time, eventmodel.page, eventmodel.description, eventmodel.updated_at 
FROM eventmodel ORDER BY eventmodel.updated_at ASC
 LIMIT 10

SELECT eventmodel.id, eventmodel.time, eventmodel.page, eventmodel.description, eventmodel.updated_at 
FROM eventmodel ORDER BY eventmodel.updated_at ASC
 LIMIT :param_1


In [ ]:
from timescaledb.hyperfunctions import time_bucket

with Session(engine) as session:
    bucket = time_bucket("1 day", EventModel.time)
    query = (
        select(
            bucket,
            EventModel.page
            
        )
        .order_by(EventModel.updated_at.asc())
    )
    compiled_query = query.compile(compile_kwargs={"literal_binds": True})
    print(compiled_query)
    print("")
    print(str(query))